# Corrupt Record Handling

Apache Spark&trade; and Azure Databricks&reg; provide ways to handle corrupt records.

-sandbox
## Working with Corrupt Data

ETL pipelines need robust solutions to handle corrupt data. This is because data corruption scales as the size of data and complexity of the data application grow. Corrupt data includes:  
<br>
* Missing information
* Incomplete information
* Schema mismatch
* Differing formats or data types
* User errors when writing data producers

Since ETL pipelines are built to be automated, production-oriented solutions must ensure pipelines behave as expected. This means that **data engineers must both expect and systematically handle corrupt records.**

In the roadmap for ETL, this is the **Handle Corrupt Records** step:
<img src="https://files.training.databricks.com/images/eLearning/ETL-Part-1/ETL-Process-3.png" style="border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa"/>

Run the following cell to mount the data:

In [4]:
%run "./Includes/Classroom-Setup"

-sandbox
Run the following cell, which contains a corrupt record, `{"a": 1, "b, "c":10}`:

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> This is not the preferred way to make a DataFrame.  This code allows us to mimic a corrupt record you might see in production.

In [6]:
data = """{"a": 1, "b":2, "c":3}|{"a": 1, "b":2, "c":3}|{"a": 1, "b, "c":10}""".split('|')

corruptDF = (spark.read
  .option("mode", "PERMISSIVE")
  .option("columnNameOfCorruptRecord", "_corrupt_record")
  .json(sc.parallelize(data))
)
display(corruptDF)  

_corrupt_record,a,b,c
null,1,2,3
null,1,2,3
"{""a"": 1, ""b, ""c"":10}",null,null,null


In the previous results, Spark parsed the corrupt record into its own column and processed the other records as expected. This is the default behavior for corrupt records, so you didn't technically need to use the two options `mode` and `columnNameOfCorruptRecord`.

There are three different options for handling corrupt records [set through the `ParseMode` option](https://github.com/apache/spark/blob/master/sql/catalyst/src/main/scala/org/apache/spark/sql/catalyst/util/ParseMode.scala#L34):

| `ParseMode` | Behavior |
|-------------|----------|
| `PERMISSIVE` | Includes corrupt records in a "_corrupt_record" column (by default) |
| `DROPMALFORMED` | Ignores all corrupted records |
| `FAILFAST` | Throws an exception when it meets corrupted records |

The following cell acts on the same data but drops corrupt records:

In [8]:
data = """{"a": 1, "b":2, "c":3}|{"a": 1, "b":2, "c":3}|{"a": 1, "b, "c":10}""".split('|')

corruptDF = (spark.read
  .option("mode", "DROPMALFORMED")
  .json(sc.parallelize(data))
)
display(corruptDF)

a,b,c
1,2,3
1,2,3


The following cell throws an error once a corrupt record is found, rather than ignoring or saving the corrupt records:

In [10]:
data = """{"a": 1, "b":2, "c":3}|{"a": 1, "b":2, "c":3}|{"a": 1, "b, "c":10}""".split('|')

corruptDF = (spark.read
  .option("mode", "FAILFAST")
  .json(sc.parallelize(data))
)
display(corruptDF)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2860209305808469> in <module> 
 3 corruptDF = (spark.read
 4 . option ( "mode" , "FAILFAST" ) 
 ----> 5 . json ( sc . parallelize ( data ) ) 
 6 )
 7 display ( corruptDF ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in json (self, path, schema, primitivesAsString, prefersDecimal, allowComments, allowUnquotedFieldNames, allowSingleQuotes, allowNumericLeadingZero, allowBackslashEscapingAnyCharacter, mode, columnNameOfCorruptRecord, dateFormat, timestampFormat, multiLine, allowUnquotedControlChars, lineSep, samplingRatio, dropFieldIfAllNull, encoding) 
 284 keyed . _bypass_serializer = True 
 285 jrdd = keyed . _jrdd . map ( self . _spark . _jvm . BytesToString ( ) ) 
 --> 286 return self . _df ( self . _jreader . json ( jrdd ) ) 
 287 else : 
 288 raise TypeError ( "path can be only string, list or RDD" ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o329.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 471.0 failed 4 times, most recent failure: Lost task 7.3 in stage 471.0 (TID 11887, 10.139.64.4, executor 3): org.apache.spark.SparkException: Malformed records are detected in schema inference. Parse Mode: FAILFAST.
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1.apply(JsonInferSchema.scala:66)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1.apply(JsonInferSchema.scala:53)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:441)
	at scala.collection.Iterator$class.isEmpty(Iterator.scala:331)
	at scala.collection.AbstractIterator.isEmpty(Iterator.scala:1334)
	at scala.collection.TraversableOnce$class.reduceLeftOption(TraversableOnce.scala:203)
	at scala.collection.AbstractIterator.reduceLeftOption(Iterator.scala:1334)
	at scala.collection.TraversableOnce$class.reduceOption(TraversableOnce.scala:210)
	at scala.collection.AbstractIterator.reduceOption(Iterator.scala:1334)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1.apply(JsonInferSchema.scala:70)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1.apply(JsonInferSchema.scala:50)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:830)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:830)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:60)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:353)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:317)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:140)
	at org.apache.spark.scheduler.Task.run(Task.scala:113)
	at org.apache.spark.executor.Executor$TaskRunner$$anonfun$13.apply(Executor.scala:533)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1541)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:539)
	at java.util.conc

### Recommended Pattern: `badRecordsPath`

Databricks Runtime has [a built-in feature](https://docs.azuredatabricks.net/spark/latest/spark-sql/handling-bad-records.html#handling-bad-records-and-files) that saves corrupt records to a given end point. To use this, set the `badRecordsPath`.

This is a preferred design pattern since it persists the corrupt records for later analysis even after the cluster shuts down.

In [12]:
data = """{"a": 1, "b":2, "c":3}|{"a": 1, "b":2, "c":3}|{"a": 1, "b, "c":10}""".split('|')

corruptDF = (spark.read
  .option("badRecordsPath", "/tmp/badRecordsPath")
  .json(sc.parallelize(data))
)
display(corruptDF)

a,b,c
1,2,3
1,2,3


-sandbox
See the results in `/tmp/badRecordsPath`.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Recall that `/tmp` is a directory backed by S3 or the Azure Blob available to all clusters.

In [14]:
display(spark.read.json("/tmp/badRecordsPath/*/*/*"))

reason,record
"com.fasterxml.jackson.core.JsonParseException: Unexpected character ('c' (code 99)): was expecting a colon to separate field name and value at [Source: {""a"": 1, ""b, ""c"":10}; line: 1, column: 16]","{""a"": 1, ""b, ""c"":10}"
"com.fasterxml.jackson.core.JsonParseException: Unexpected character ('c' (code 99)): was expecting a colon to separate field name and value at [Source: {""a"": 1, ""b, ""c"":10}; line: 1, column: 16]","{""a"": 1, ""b, ""c"":10}"
"com.fasterxml.jackson.core.JsonParseException: Unexpected character ('c' (code 99)): was expecting a colon to separate field name and value at [Source: {""a"": 1, ""b, ""c"":10}; line: 1, column: 16]","{""a"": 1, ""b, ""c"":10}"


## Exercise 1: Working with Corrupt Records

### Step 1: Diagnose the Problem

Import the data used in the last lesson, which is located at `/mnt/training/UbiqLog4UCI/14_F/log*`.  Import the corrupt records in a new column `SMSCorrupt`.  <br>

Save only the columns `SMS` and `SMSCorrupt` to the new DataFrame `SMSCorruptDF`.

In [17]:
# TODO
SMSCorruptDF = (spark.read
  .json('/mnt/training/UbiqLog4UCI/14_F/log*')
).select("SMS", "SMSCorrupt")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o553.select.
: org.apache.spark.sql.AnalysisException: cannot resolve '`SMSCorrupt`' given input columns: [Bluetooth, SMS, Location, WiFi, _corrupt_record, Application, Call];;
'Project [SMS#5983, 'SMSCorrupt]
+- Relation[Application#5979,Bluetooth#5980,Call#5981,Location#5982,SMS#5983,WiFi#5984,_corrupt_record#5985] json

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:120)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:122)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpressionsUp(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1.apply(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1.apply(CheckAnalysis.scala:89)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:147)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$class.checkAnalysis(CheckAnalysis.scala:89)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$e

In [18]:
# TEST - Run this cell to test your solution
cols = set(SMSCorruptDF.columns)
SMSCount = SMSCorruptDF.cache().count()

dbTest("ET1-P-06-01-01", {'SMS', 'SMSCorrupt'}, cols)
dbTest("ET1-P-06-01-02", 8, SMSCount)

print("Tests passed!")

-sandbox
Examine the corrupt records to determine what the problem is with the bad records.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** Take a look at the name in metadata.

The entry `{"name": "mr Khojasteh"flash""}` should have single quotes around `flash` since the double quotes are interpreted as the end of the value.  It should read `{"name": "mr Khojasteh'flash'"}` instead.

The optimal solution is to fix the initial producer of the data to correct the problem at its source.  In the meantime, you could write ad hoc logic to turn this into a readable field.

### Step 2: Use `badRecordsPath`

Use the `badRecordsPath` option to save corrupt records to the directory `/tmp/corruptSMS`.

In [22]:
# TODO

In [23]:
# TEST - Run this cell to test your solution
SMSCorruptDF2.count()
corruptCount = spark.read.json("/tmp/corruptSMS/*/*/*").count()

dbTest("ET1-P-06-02-01", True, corruptCount >= 8)

print("Tests passed!")

## Review
**Question:** By default, how are corrupt records dealt with using `spark.read.json()`?  
**Answer:** They appear in a collumn called `_corrupt_record`.

**Question:** How can a query persist corrupt records in separate destination?  
**Answer:** The Databricks feature `badRecordsPath` allows a query to save corrupt records to a given end point for the pipeline engineer to investigate corruption issues.

## Next Steps

Start the next lesson, [Loading Data and Productionalizing]($./07-Loading-Data-and-Productionalizing).

## Additional Topics & Resources

**Q:** Where can I get more information on dealing with corrupt records?  
**A:** Check out the Spark Summit talk on <a href="https://databricks.com/session/exceptions-are-the-norm-dealing-with-bad-actors-in-etl" target="_blank">Exceptions are the Norm: Dealing with Bad Actors in ETL</a>